In [1]:
import torch
import torch.utils.benchmark as benchmark

from alibi import ALiBiAttentionMaskGenerator
from metaseq import buffered_future_mask

In [2]:
seq_len = 512
batch_size = 1

# The num_heads is hard coded inside buffered_future_mask as 8
# Metaseq's buffered_future_mask is a class method, num_heads is 
# taken from a class attribute.
x = torch.empty((seq_len, batch_size))
t0 = benchmark.Timer(
    stmt = "buffered_future_mask(x)",
    globals = {
        "buffered_future_mask": buffered_future_mask,
        "x": x,
        },
    label = "Metaseq")


# model_size is abitrary for alibi,
# but it's the expected dimension for X in fairseq's 2 AttentionMask
model_size = 1
x = torch.empty((seq_len, model_size))
t1 = benchmark.Timer(
    stmt = """
        alibi_gen = ALiBiAttentionMaskGenerator(num_heads=8)
        alibi_gen(x)
        """,
    globals = {
        "ALiBiAttentionMaskGenerator": ALiBiAttentionMaskGenerator,
        "x": x,
        },
    label = "Fairseq2")

In [3]:
res0 = t0.timeit(30000)
res1 = t1.timeit(30000)
print(res0)
print(res1)

[W ParallelNative.cpp:230] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


Metaseq
  845.30 us
  1 measurement, 30000 runs , 1 thread
Fairseq2
  726.37 us
  1 measurement, 30000 runs , 1 thread
